# Conversion of Raw 2-Photon Imaging Data for Sharing with Annotations

## 1. Let's begin with importing the config files

In [1]:
# base and metaprogramming imports
from __future__ import annotations
import os
import sys
import itertools as it
import pprint as pp
from typing import Optional
import warnings

# analysis imports
import numpy as np
import pandas as pd

# visualization imports
import napari
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# ipynb-specific imports
try:
    import ipynbname
except Exception as e:
    warnings.warn(f'`ipynbname` module import failed with error: {e!r}')
    ipynbname = None
    
if ipynbname is not None:
    _file = ipynbname.path()
else:
    _file = ''
    
jobs_dir = os.path.split(_file)[0]
sources_dir = os.path.abspath(os.path.join(jobs_dir, '..'))
repo_dir = os.path.abspath(os.path.join(sources_dir, '..'))
config_dir = os.path.abspath(os.path.join(repo_dir, 'configs'))

# add sources directory to path
if sources_dir not in sys.path:
    sys.path.append(sources_dir)

# local imports
import c_swain_python_utils as csutils
from utilities import *
import imaging_dataset as imd
import prairie_view_imports as pvi

_name = csutils.no_ext_basename(_file)

In [2]:
# plotting defaults
sns.set_style('ticks')

In [3]:
# logging setup
log = csutils.get_logger(_name)

debug_mode = False
default_window_level = 'debug'

log_path = os.path.join(sources_dir, '..', 'logs', f'{_file}.log')
csutils.apply_standard_logging_config(
    file_path=log_path,
    window_level='debug' if debug_mode else default_window_level,
    window_format='debug' if debug_mode else 'default')

In [4]:
config_list = [
    'cs-ii-29_config.yml',
    'cs-ii-33_config.yml',
    'cs-ii-34_config.yml']

config_list = [ZMIAConfig(os.path.join(config_dir, c)) 
               for c in config_list]

all_dataset_infos = sum([c.dataset_list for c in config_list], [])
all_2p_live_dataset_infos = [
    di for di in all_dataset_infos                    
    if (di.type == 'two-photon' and di.run_id is not None)]

num_matched_datasets = len(all_2p_live_dataset_infos)
log.info('Matched {:d} live datasets.', num_matched_datasets)

local-nb_cs-ii-[29,33,34]_live-2p-preparation (  15): INFO : Matched 16 live datasets.


In [5]:
datasets: list[Optional[imd.ImagingDataset]] = []
dataset_info: ZMIADatasetInfo
for i, dataset_info in enumerate(all_2p_live_dataset_infos):
    log.info('>'*15 + ' Loading Dataset {:d} of {:d} ' + '<'*15, i+1, num_matched_datasets)
    try:
        imaging_data = imd.ImagingDataset.from_dataset_info(dataset_info)
    except Exception as e:
        imaging_data = None
        log.error('Failed to load dataset. Failed with error:', exc_info=e)
    else:
        log.info(f'Loaded {imaging_data}')        
        imaging_data.clear_cache(disk=False)
    
    datasets.append(imaging_data)

local-nb_cs-ii-[29,33,34]_live-2p-preparation (   4): INFO : >>>>>>>>>>>>>>> Loading Dataset 1 of 16 <<<<<<<<<<<<<<<
prairie_view_imports (2745): INFO : Loading dataset (two-photon-fish-29B_run-395_2p-tz) from config cs-ii-29_all.
prairie_view_imports (1534): INFO : Attempting to load dataset from cache.
prairie_view_imports (1551): INFO : Loading of dataset from cache was successful.
prairie_view_imports (1564): INFO : Loading in cached images.
prairie_view_imports (1572): INFO : Loading in of cached images is sucessful.
imaging_dataset (1051): INFO : Categorized PVDataset as PVDatasetCategory.Z_STACK_IMAGE|TZ_SERIES_IMAGE|MULTI_SEQUENCE
imaging_dataset (1058): INFO : Initializing ImagingDataset(s) from prairie view dataset PVDataset(name='two-photon-fish-29B_run-395_2p-tz', num_sequences=855)
imaging_dataset ( 699): INFO : Attempting to load imaging dataset from cache.
local-nb_cs-ii-[29,33,34]_live-2p-preparation (  11): INFO : Loaded ImagingDataset(name='two-photon-fish-29B_run-395

prairie_view_imports (1553): ERROR : Initialization of dataset from cache failed.
Traceback (most recent call last):
  File "I:\CorbanSwain\repos\zmia\source\prairie_view_imports.py", line 1539, in _init_from_cache
    cache_object: _PVDatasetCache = pickle.load(_file)
_pickle.UnpicklingError: pickle data was truncated


prairie_view_imports (1556): INFO : Falling back to initialize dataset from xml.
prairie_view_imports (1492): INFO : Importing dataset metadata from xml.
prairie_view_imports (1621): INFO : Caching dataset to disk.
prairie_view_imports (1655): INFO : Loading up to 51018 2D image(s) from 1546 image files on disk; this may take some time.
prairie_view_imports (1670): INFO : Loading images using 32 parallel processes.
prairie_view_imports (1698): INFO : All images loaded.


local-nb_cs-ii-[29,33,34]_live-2p-preparation (   9): ERROR : Failed to load dataset. Failed with error:
Traceback (most recent call last):
  File "C:\Users\CorbanSwain\AppData\Local\Temp\ipykernel_21152\3884351420.py", line 6, in <module>
    imaging_data = imd.ImagingDataset.from_dataset_info(dataset_info)
  File "I:\CorbanSwain\repos\zmia\source\imaging_dataset.py", line 858, in from_dataset_info
    return cls.from_dataset(loader(dataset_info=dataset_info),
  File "I:\CorbanSwain\repos\zmia\source\prairie_view_imports.py", line 2749, in load_dataset
    return PVDataset(dataset_info.full_path, **kwargs)
  File "I:\CorbanSwain\repos\zmia\source\prairie_view_imports.py", line 1386, in __init__
    self._init_from_path(path,
  File "I:\CorbanSwain\repos\zmia\source\prairie_view_imports.py", line 1485, in _init_from_path
    init_function(**init_func_kwargs)
  File "I:\CorbanSwain\repos\zmia\source\prairie_view_imports.py", line 1562, in _init_from_cache
    if cache_object.images is n

local-nb_cs-ii-[29,33,34]_live-2p-preparation (   4): INFO : >>>>>>>>>>>>>>> Loading Dataset 9 of 16 <<<<<<<<<<<<<<<
prairie_view_imports (2745): INFO : Loading dataset (two-photon-ref-cs-ii-34_fish-A_run-202_2p-tz) from config cs-ii-34.
prairie_view_imports (1492): INFO : Importing dataset metadata from xml.
prairie_view_imports (1621): INFO : Caching dataset to disk.
prairie_view_imports (1655): INFO : Loading up to 31720 2D image(s) from 793 image files on disk; this may take some time.
prairie_view_imports (1670): INFO : Loading images using 32 parallel processes.
prairie_view_imports (1698): INFO : All images loaded.
imaging_dataset (1051): INFO : Categorized PVDataset as PVDatasetCategory.Z_STACK_IMAGE|TZ_SERIES_IMAGE|MULTI_SEQUENCE
imaging_dataset (1058): INFO : Initializing ImagingDataset(s) from prairie view dataset PVDataset(name='two-photon-ref-cs-ii-34_fish-A_run-202_2p-tz', num_sequences=793)
imaging_dataset ( 699): INFO : Attempting to load imaging dataset from cache.
ima

local-nb_cs-ii-[29,33,34]_live-2p-preparation (   9): ERROR : Failed to load dataset. Failed with error:
Traceback (most recent call last):
  File "C:\Users\CorbanSwain\AppData\Local\Temp\ipykernel_21152\3884351420.py", line 6, in <module>
    imaging_data = imd.ImagingDataset.from_dataset_info(dataset_info)
  File "I:\CorbanSwain\repos\zmia\source\imaging_dataset.py", line 858, in from_dataset_info
    return cls.from_dataset(loader(dataset_info=dataset_info),
  File "I:\CorbanSwain\repos\zmia\source\imaging_dataset.py", line 867, in from_dataset
    return cls.from_pvdataset(dataset, **kwargs)
  File "I:\CorbanSwain\repos\zmia\source\imaging_dataset.py", line 1173, in from_pvdataset
    assert len(single_2d_image) == 1, (
AssertionError: Expected to receive a scalar PVDataList here; got PVDataList of len 0 instead.


In [6]:
for imaging_data in datasets:
    # objective
    metadata_dict = imaging_data.source_data.unique_metadata
    obj_lens = metadata_dict.get_value(pvi.pvmk.OBJECTIVE_LENS)
    
    # bidirectional scan
    is_bidir_scan = metadata_dict.get_value(pvi.pvmk.BIDIRECTIONAL_Z_SCAN)
    
    # x-pitch-um, y-pitch-um
    x_pitch_um = metadata_dict.get_value((pvi.pvmk.MICRONS_PER_PIXEL, pvi.pvmk.X_AXIS))
    y_pitch_um = metadata_dict.get_value((pvi.pvmk.MICRONS_PER_PIXEL, pvi.pvmk.Y_AXIS))
    
    # z-pitch-um
    z_pitch_um = imaging_data.coordinates["z"].median_pitch
    
    # volume-acquisition-rate-hz
    acquisition_rate_hz = 1 / imaging_data.coordinates["time"].median_pitch
    
    # slice data if it's not a bidirectional scan
    if is_bidir_scan:
        imaging_data_slice = imaging_data
    else:
        imaging_data_slice = imaging_data.drop_slices([0, 1], imd.Dim.Z)        
    
    # shape
    shape_dict = imaging_data_slice.shape_dict
    
    select_metadata_dict = {
        'x-pitch_um': x_pitch_um,
        'y-pitch_um': y_pitch_um,
        'z-pitch_um': z_pitch_um,
        'acquisition-rate_hz': acquisition_rate_hz,
        'is-bidirectional-z-scan': is_bidir_scan,
        'shape_unitless': {
            'x': shape_dict[Dim.X],
            'y': shape_dict[Dim.Y],
            'z': shape_dict[Dim.Z],
            'time': shape_dict[Dim.TIME]},
        'objective-lens': obj_lens
    }
    
    pp.pprint(select_metadata_dict)
    break

AttributeError: 'ImagingDataset' object has no attribute '_get_image_data_func'

In [ ]:
# metadata experiments
# shape
imaging_data.shape_dict
log.info(f'{imaging_data_slice.shape_dict = }')

# objective
metadata_dict = imaging_data.source_data.unique_metadata
log.info(f'{metadata_dict.get_value(pvi.pvmk.OBJECTIVE_LENS) = }')

# bidirectional scan
log.info(f'{metadata_dict.get_value(pvi.pvmk.BIDIRECTIONAL_Z_SCAN) = }')

# x-pitch-um, y-pitch-um
log.info(f'{metadata_dict.get_value((pvi.pvmk.MICRONS_PER_PIXEL, pvi.pvmk.X_AXIS)) = }')
log.info(f'{metadata_dict.get_value((pvi.pvmk.MICRONS_PER_PIXEL, pvi.pvmk.Y_AXIS)) = }')

# z-pitch-um
log.info(f'{imaging_data.coordinates["z"].median_pitch = }')

# volume-acquisition-rate-hz
log.info(f'{1 / imaging_data.coordinates["time"].median_pitch = }')